In [6]:
from pyspark.sql.session import SparkSession
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd
from pyspark.sql.types import *
from pyspark.sql.functions import concat,concat_ws
from pyspark.sql.functions import *
import re
import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
# nltk.download('punkt')
# nltk.download('stopwords')
from nltk.stem import WordNetLemmatizer
# nltk.download('wordnet')
import pandas as pd
import sklearn as sk
import numpy as np
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [7]:
product_path='D:/PROJECTS/FASHIONPASS/Database/product.csv'
tagcloud_path='D:/PROJECTS/FASHIONPASS/Database/tag_cloud.csv'
collection_category_tag_path='D:/PROJECTS/FASHIONPASS/Database/collection_category_tag.csv'
product_tag_path='D:/PROJECTS/FASHIONPASS/Database/product_tag.csv'
features_path='D:/PROJECTS/FASHIONPASS/Database/spars_dense_features.csv'
collection_category_path='D:/PROJECTS/FASHIONPASS/Database/collection_category.csv'
clothing_collection_path='D:/PROJECTS/FASHIONPASS/Database/cloting_collection_category.csv'
product_cloth_collection_path='D:/PROJECTS/FASHIONPASS/Database/product_cloth_cloection_tag.csv'

In [8]:
spark = SparkSession.builder \
    .appName("PySpark MySQL Example") \
    .config("spark.driver.extraClassPath", "C:/spark/spark-3.5.0-bin-hadoop3/jars/mysql-connector-java-5.1.46.jar") \
    .getOrCreate()



In [9]:
Product_df=spark.read.format('csv').option('header',True) \
.load(product_path)

Tagcloud_df=spark.read.format('csv').option('header',True) \
.load(tagcloud_path)


Collection_Category_tag_df=spark.read.format('csv').option('header',True) \
.load(collection_category_tag_path)

Product_tag_df=spark.read.format('csv').option('header',True) \
.load(product_tag_path)


features_df=spark.read.format('csv').option('header',True) \
.load(features_path)


Collection_Category_df=spark.read.format('csv').option('header',True)\
.load(collection_category_path)

clothing_collection_df=spark.read.format('csv').option('header',True)\
.load(clothing_collection_path)

# product_cloth_collection_tag_df=spark.read.format('csv').option('header',True)\
# .load(product_cloth_collection_path)

In [10]:
product_title_string=[ string[0].lower() for string in Product_df.select('product_title').collect()]
product_tag_id=[ id[0] for id in Product_df.select('product_id').collect()]

In [11]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('bert-base-nli-mean-tokens')

# Encode sentences
embeddings = model.encode(product_title_string, convert_to_tensor=True)

c:\Users\Z2 mini\.conda\envs\RS\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
cos_sim = util.cos_sim(embeddings, embeddings)

cos_sim

tensor([[1.0000, 0.3454, 0.3454,  ..., 0.3033, 0.4007, 0.4427],
        [0.3454, 1.0000, 1.0000,  ..., 0.5798, 0.5492, 0.3246],
        [0.3454, 1.0000, 1.0000,  ..., 0.5798, 0.5492, 0.3246],
        ...,
        [0.3033, 0.5798, 0.5798,  ..., 1.0000, 0.5118, 0.2174],
        [0.4007, 0.5492, 0.5492,  ..., 0.5118, 1.0000, 0.4157],
        [0.4427, 0.3246, 0.3246,  ..., 0.2174, 0.4157, 1.0000]])

In [13]:
all_product_combination = {'cosin_score_title':[] , 'Product_A':[] , 'Product_B':[]}
for i in range(len(cos_sim)-1):
    for j in range(i+1, len(cos_sim)):
        all_product_combination['cosin_score_title'].append(float(cos_sim[i][j]))
        all_product_combination['Product_A'].append(product_tag_id[i])
        all_product_combination['Product_B'].append(product_tag_id[j])

In [14]:
recomendation_df=pd.DataFrame(all_product_combination)

In [27]:
recomendation_df.to_csv('D:/PROJECTS/FASHIONPASS/cosin_similarity_registory/cosine_similarity_product_title_1_9_2023.csv')

In [39]:
product_title='ROCKY RIDGE JACKET'
selected_product_id=Product_df.filter(Product_df['product_title']==product_title).select('product_id').collect()[0][0]
recomendation_df_product=recomendation_df[recomendation_df['Product_A']==selected_product_id]
link=[]
for index,row in recomendation_df_product.iterrows():
    if row['cosin_score_title']>=.20:
        id=row['Product_B']
        # link.append(int(id))
        # product_thumbnail=Product_df.filter(Product_df.product_id==id).select('product_thumbnail').collect()
        try:
            # print('https://fashionpass.s3.us-west-1.amazonaws.com/products/'+(product_thumbnail[0][0]))
             link.append(id)
        except:
            print('valid string')    
print(','.join(link))

6488,6489,6490,6491,6492,6493,6494,6495,6496,6497,6498,6499,6500,6501,6502,6503,6504,6505,6506,6507,6508,6509,6510,6511,6512,6513,6514,6515,6516,6517,6518,6519,6520,6521,6522,6525,6526,6527,6528,6529,6530,6531,6532,6533,6534,6535,6536,6538,6539,6540,6541,6542,6543,6544,6545,6546,6547,6548,6550,6551,6552,6553,6554,6555,6556,6557,6558,6559,6560,6561,6562,6563,6564,6565,6566,6567,6568,6569,6570,6572,6573,6574,6575,6576,6577,6578,6579,6580,6581,6582,6583,6584,6585,6586,6587,6588,6589,6590,6591,6592,6593,6594,6595,6596,6597,6598,6599,6600,6601,6602,6604,6606,6607,6608,6609,6610,6611,6612,6613,6614,6615,6616,6617,6618,6619,6620,6621,6622,6623,6624,6625,6626,6627,6629,6631,6632,6636,6638,6639,6640,6641,6642,6643,6644,6645,6646,6647,6648,6649,6650,6651,6652,6653,6654,6655,6657,6658,6659,6660,6661,6662,6663,6664,6665,6666,6667,6668,6669,6670,6671,6672,6674,6675,6676,6677,6678,6679,6680,6681,6682,6683,6684,6685,6686,6688,6689,6690,6691,6692,6693,6694,6695,6696,6697,6698,6699,6700,6701,6702,6703,

In [36]:
recomendation_df_product

,cosin_score_title,Product_A,Product_B
36487850,0.288301,7641,7642
36487851,0.298659,7641,7643
36487852,0.379354,7641,7644
36487853,0.484141,7641,7645
36487854,0.489142,7641,7646
...,...,...,...
36489052,0.423794,7641,8868
36489053,0.291308,7641,8869
36489054,0.295560,7641,8870
36489055,0.467187,7641,8871
